In [2]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam

In [3]:
# 加載 MNIST 數據
(x_train1, y_train1), (x_test1, y_test1) = mnist.load_data()

# 正規化並重塑數據
# x_train = x_train1[0:100].astype('float32') / 255.
y_test = y_test1 [0:100]
x_test = x_test1[0:100].astype('float32') / 255.
x_train = x_train1[0:5000].astype('float32') / 255.
# x_test = x_test1.astype('float32') / 255.

print(x_train.shape)
print(x_test.shape)

(5000, 28, 28)
(100, 28, 28)


In [ ]:
# n = 10  # 顯示幾個數字
# plt.figure(figsize=(20, 4))
# for i in range(n):
#     # 顯示原始數據
#     ax = plt.subplot(2, n, i + 1)
#     plt.imshow(x_test[i].reshape(28, 28))
#     plt.gray()
#     ax.get_xaxis().set_visible(False)
#     ax.get_yaxis().set_visible(False)

# plt.show()

In [4]:
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from tensorflow.keras.layers import Dense, Flatten, Reshape,Cropping2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

input_shape = (28, 28, 1)  # MNIST图像的尺寸和通道数

# 编碼器
encoder_input = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(encoder_input)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Flatten()(x)  # Flatten the output
x = Dense(128, activation='relu')(x)  # Additional ANN layer
latent_vector = Dense(64, activation='relu')(x)  # Latent vector

# 解碼器
decoder_input = Dense(128, activation='relu')(latent_vector)
x = Dense(8 * 4 * 4, activation='relu')(decoder_input)  # 8 is the number of filters, 4x4 is the feature map size before flattening
x = Reshape((4, 4, 8))(x)  # Reshape into the shape before the last MaxPooling2D
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2DTranspose(32, (3, 3), strides=(2,2), activation='relu', padding='same')(x)
x = Cropping2D(cropping=((2, 2), (2, 2)))(x)  # 如果需要，可以添加裁剪层来精确调整尺寸
decoded_output = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

# 自编碼器模型
autoencoder = Model(inputs=encoder_input, outputs=decoded_output)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# 顯示模型结構
autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 16)        4624      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 16)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 7, 7, 8)           1160  

In [5]:
from tensorflow.keras.callbacks import ModelCheckpoint
model_checkpoint = ModelCheckpoint('UNet_Membrane.hdf5', monitor='loss',verbose=1, save_best_only=True)
history = autoencoder.fit(x_train, x_train, 
                epochs=10, 
                batch_size=128, 
                shuffle=True, 
                callbacks=[model_checkpoint],
                validation_data=(x_test, x_test))


Epoch 1/10
40/40 [==============================] - ETA: 0s - loss: 0.4649
Epoch 1: loss improved from inf to 0.46486, saving model to UNet_Membrane.hdf5
40/40 [==============================] - 10s 226ms/step - loss: 0.4649 - val_loss: 0.2935
Epoch 2/10
40/40 [==============================] - ETA: 0s - loss: 0.2805
Epoch 2: loss improved from 0.46486 to 0.28046, saving model to UNet_Membrane.hdf5
40/40 [==============================] - 10s 247ms/step - loss: 0.2805 - val_loss: 0.2559
Epoch 3/10
39/40 [============================>.] - ETA: 0s - loss: 0.2553
Epoch 3: loss improved from 0.28046 to 0.25531, saving model to UNet_Membrane.hdf5
40/40 [==============================] - 10s 259ms/step - loss: 0.2553 - val_loss: 0.2386
Epoch 4/10
40/40 [==============================] - ETA: 0s - loss: 0.2348
Epoch 4: loss improved from 0.25531 to 0.23478, saving model to UNet_Membrane.hdf5
40/40 [==============================] - 10s 255ms/step - loss: 0.2348 - val_loss: 0.2195
Epoch 5/10
4

In [6]:
# autoencoder.compile(optimizer=Adam(), loss='binary_crossentropy')
# history = autoencoder.fit(x_train, x_train, 
#                 epochs=200, 
#                 batch_size=256, 
#                 shuffle=True, 
#                 validation_data=(x_test, x_test))


In [7]:
# encoded_imgs = encoder.predict(x_test)
# decoded_imgs = decoder.predict(encoded_imgs)
decoded_imgs = autoencoder.predict(x_test)


4/4 [==============================] - 1s 53ms/step


In [ ]:
# comparing the original image and decoder image
n = 10  # 顯示幾個數字
plt.figure(figsize=(20, 4))
for i in range(n):
    # 顯示原始數據
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

#     顯示重建數據
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [9]:
# 假设 x_test 是你的测试数据集
encoded_imgs = encoder.predict(x_test)

NameError: name 'encoder' is not defined

In [ ]:
from sklearn.decomposition import PCA

# 初始化 PCA，设置降维后的主成分数为2
pca = PCA(n_components=2)
encoded_imgs_2d = pca.fit_transform(encoded_imgs)  # 对编码后的图片应用PCA


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))
scatter = plt.scatter(encoded_imgs_2d[:, 0], encoded_imgs_2d[:, 1], c=y_test, cmap='CMRmap_r', alpha=0.5)
plt.colorbar(scatter)  # 显示颜色条
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.title('PCA of Encoded Images with Labels')


NameError: name 'encoded_imgs_2d' is not defined